# Experimenting with Chatbots!
- Part 1: Rule based chatbots
- Part 2: Understanding natural language
- Part 3: Building a virtual assistant
- Part 4: Dialogue

<font color=green size=5> <b>1. Basics for rule-based CBs</font>

In [198]:
import time
import numpy as np
import re
import spacy

### 1.1 Set basic send_message function and rules for responses

In [199]:
#Set basic send_message function
bot_template = "BOT : {0}"
user_template = "USER : {0}"

# Define a function that sends a message to the bot: send_message
def send_message(message):
    # Print user_template including the user_message
    print(user_template.format(message))
    # Get the bot's response to the message
    response = respond(message)
    # Print the bot template including the bot's response.
    time.sleep(1) #add response delay for authenticity!!!
    print(bot_template.format(response))

### 1.2 Substituting Pronouns

In [200]:
# Define replace_pronouns()
def replace_pronouns(message):

    message = message.lower()
    if 'me' in message:
        # Replace 'me' with 'you'
        return re.sub('me', 'you', message)
    if 'my' in message:
        # Replace 'my' with 'your'
        return re.sub('my', 'your', message)
    if 'your' in message:
        # Replace 'your' with 'my'
        return re.sub('your', 'my', message)
    if 'you' in message:
        # Replace 'you' with 'me'
        return re.sub('you', 'me', message)

    return message

print(replace_pronouns("my last birthday"))
print(replace_pronouns("when you went to Florida"))
print(replace_pronouns("I had my own castle"))

your last birthday
when me went to florida
i had your own castle


### 1.3 Basic intent matching

In [201]:
#Set base variables for responses!!!
responses = {'default': 'default message',
 'goodbye': 'goodbye for now',
 'greet': 'Hello you! :)',
 'thankyou': 'you are very welcome'}

patterns = {'goodbye': re.compile(r'bye|farewell', re.UNICODE),
 'greet': re.compile(r'hello|hi|hey', re.UNICODE),
 'thankyou': re.compile(r'thank|thx', re.UNICODE)}

# Define a function to find the intent of a message
def match_intent(message):
    matched_intent = None
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if pattern.search(message):
            matched_intent = intent
    return matched_intent

# Define a respond function
def respond(message):
    # Call the match_intent function
    intent = match_intent(message)
    # Fall back to the default response
    key = "default"
    if intent in responses:
        key = intent
    return responses[key]

# Send messages
send_message("hello!")
send_message("bye byeee")
send_message("thanks very much!")

USER : hello!
BOT : Hello you! :)
USER : bye byeee
BOT : goodbye for now
USER : thanks very much!
BOT : you are very welcome


### 1.4 Basic name identificaiton

In [202]:
# Define find_name()
def find_name(message):
    name = None
    # Create a pattern for checking if the keywords occur
    name_keyword = re.compile('name|call')
    # Create a pattern for finding capitalized words
    name_pattern = re.compile('[A-Z]{1}[a-z]*')
    if name_keyword.search(message):
        # Get the matching words in the string
        name_words = name_pattern.findall(message)
        if len(name_words) > 0:
            # Return the name if the keywords are present
            name = ' '.join(name_words)
    return name

# Define respond()
def respond(message):
    # Find the name
    name = find_name(message)
    if name is None:
        return "Hi there!"
    else:
        return "Hello, {0}!".format(name)

# Send messages
send_message("my name is Hogward The Intruder")

USER : my name is Hogward The Intruder
BOT : Hello, Hogward The Intruder!


<font color=green size=5> <b>2. Understanding Natural Language </font>

### 2.1 Word vectors
- Word vectors assign to each word, a vector that describes its meaning
- words that appear in similar contexts often will have similar word vectors
- If you create these vectors which have BILLIONs of words, you create vectors that capture a ton of implicit meaning
- training word vectors can take a lot of computing power and a lot of data
- Fortunately, we can get pre-trained vecotrs using spacy, which is has vectors trained using GloVe (a cousin of word2vec)
- Word vectors tend to have a length of a few hundrd elements, check this with nlp.vocab.vectors_length attribute of a spacey object (see below)

#### Cosine Similarity:
- In word vector space, it is the **DIRECTION** of the word vectors that matters most. 
- the "distance" between words = angle between the vectors
- Measure of angle is **Cosine simlarity:**
    - 1 if vectors point in the same direction
    - 0 if they are perpendicular
    - -1 if they point in the opposite direciton


In [203]:
#simple word vector example
nlp = spacy.load('en') #create spacy object (loads default english model)
doc = nlp('hello, can you help me?') #produces iterator over tokens in string

In [204]:
#each vector word has its own vector
doc_total = np.ones(300,)
for token in doc:
    print("{} : {}".format(token, token.vector[:3]))

hello : [ 0.25233001  0.10176    -0.67484999]
, : [-0.082752    0.67203999 -0.14986999]
can : [-0.23857     0.35457    -0.30219001]
you : [-0.11076     0.30785999 -0.51980001]
help : [-0.29370001  0.32253    -0.44779   ]
me : [-0.15396     0.31894001 -0.54887998]
? : [-0.086864    0.19160999  0.10915   ]


In [205]:
#cosine similarity
doc_1 = nlp("cat")
doc_2 = nlp("dog")
doc_3 = nlp("fish")
doc_4 = nlp("table")
doc_5 = nlp("man")
doc_6 = nlp("woman")
doc_7 = nlp("fart")

print("Cat-Dog", doc_1.similarity(doc_2))
print("Cat-fish",doc_1.similarity(doc_3))
print("Cat-table",doc_1.similarity(doc_4))
print("man-woman",doc_5.similarity(doc_6))
print("woman-oil",doc_6.similarity(doc_7))

Cat-Dog 0.801685470553
Cat-fish 0.418065391278
Cat-table 0.285514238043
man-woman 0.74017436681
woman-oil 0.294893511308


In [206]:
nlp("cat").vector[0:5]

array([-0.15067001, -0.024468  , -0.23367999, -0.23378   , -0.18381999], dtype=float32)

In [207]:
nlp("cat dog").vector[0:5]

array([-0.27621502,  0.173051  , -0.1061995 , -0.28751498, -0.067141  ], dtype=float32)

<font color=red> To get the vector of a string of words, you take the average of the vectors of all the words in the sentence.</font>

In [208]:
(nlp("cat").vector[0:5] + nlp("dog").vector[0:5])/2

array([-0.27621502,  0.173051  , -0.1061995 , -0.28751498, -0.067141  ], dtype=float32)

### 2.2 Intents and classification

- Now that we know how to use spacy to create vector respresentations of words and documents, we can use machine learning algorithms to identify intents.
- Intent recognition is a classification problem. 
- Support vector classifiers work very well for classifying intents

**Here we will compare the accuracy of the SVC and KNN models in prediction intention:**

- <font color=red> Data is saved in the module 'datacamp_datasets.pt' in C:\Users\mciniello\AppData\Local\Continuum\anaconda3\envs\nlp_env.  
- A copy is also saved in C:\Users\mciniello\Desktop\Python\Updated projects\Clean\data. </font>

In [209]:
from datacamp_datasets4 import X_train, y_train, X_test, y_test
X_train = X_train()
y_train = y_train()
X_test = X_test()
y_test = y_test()

In [210]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(1209, 300)
(1209,)
(201, 300)


In [211]:
# Import SVC
from sklearn.svm import SVC

# Create a support vector classifier
clf = SVC(C=1)

# Fit the classifier using the training data
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Count the number of correct predictions
n_correct = 0
for i in range(len(y_test)):
    if y_pred[i] == y_test[i]:
        n_correct += 1

print("Predicted {0} correctly out of {1} test examples".format(n_correct, len(y_test)))


Predicted 162 correctly out of 201 test examples


### 2.3 Entity Extraction
- It can be quite difficult to recognize entities that the training data has not yet seen
- To generalize, we can look at how a word is spelled, which words come before or after it

**spaCy pre-built Named Entity Recognition models**
- use thesse pre-trained models (trained with A LOT of training data) to do NER on your text data

In [212]:
doc = nlp("my friend Mary has worked at Google since 2009")
for ent in doc.ents:
    print('TEXT: ', ent.text, '\tLABEL: ',ent.label_)

TEXT:  Mary 	LABEL:  PERSON
TEXT:  Google 	LABEL:  ORG
TEXT:  2009 	LABEL:  DATE


**Dependency Parsing:**

Entities can have different roles. For example:
- I want a flight from TelAviv to Toronto

So though these are both entities, one is the origin and one is a detination, and that distinction is VERY IMPORTANT. This is where dependency parsing comes in handy!

This is too big a topic to cover here (cover it yourself later!) but you can easily generate them with spacy. Heres a basic overview:
- parse  tree is a hierarchical structure that specifies parent-child relationships between the words in a phras
- it is **INDEPENDENT OF WORD ORDER**

**IN BOTH PHRASES:** "I want a flight to Shanghai from Singapore" AND "I want a flight from Singapore to Shanghai"
- "to" is the parent of Shanghai
- "from" is the parent of Singapore


![](pictures/parsetree.jpg)

In [217]:
#example1
doc = nlp('a flight to Shanghai from Singapore')
shanghai, singapore = doc[3], doc[5] #entities
list(shanghai.ancestors) #get parents of shanghai entity using .attributes

[to, flight]

**Using spaCy's entity recogniser**

In this exercise you'll use spaCy's built-in entity recognizer to extract names, dates, and organizations from search queries. Your job is to define a function called extract_entities() which takes in a single argument message and returns a dictionary with the included entity types as keys, and the extracted entities as values. The included entity types are contained in a list called include_entities.

In [220]:
dict.fromkeys(['ayyy','sup'])

{'ayyy': None, 'sup': None}

In [224]:
###########################################################################
############### EXTRACT ONLY ENTITIES YOU ARE INTERESTED IN ###############
###########################################################################

# Define included entities
include_entities = ['DATE', 'ORG', 'PERSON']

# Define extract_entities()
def extract_entities(message):
    # Create a dict to hold the entities
    ents = dict.fromkeys(include_entities)
    # Create a spacy document
    doc = nlp(message)
    for ent in doc.ents:
        if ent.label_ in include_entities:
            # Save interesting entities
            ents[ent.label_] = ent.text
    return ents

print(extract_entities('friends called Mary who have worked at Google since 2010'))
print(extract_entities('people who graduated from MIT in 1999'))
print(extract_entities('people who graduated from university in Canada in 1999'))


{'PERSON': 'Mary', 'DATE': '2010', 'ORG': 'Google'}
{'PERSON': None, 'DATE': '1999', 'ORG': 'MIT'}
{'PERSON': None, 'DATE': '1999', 'ORG': None}


In [227]:
####################################################
############### EXTRACT ALL ENTITIES ###############
####################################################

# Define extract_entities()
def extract_entities(message):
    ents = {}
    # Create a spacy document
    doc = nlp(message)
    for ent in doc.ents:
        # Save interesting entities
        ents[ent.label_] = ent.text
    return ents

print(extract_entities('friends called Mary who have worked at Google since 2010'))
print(extract_entities('people who graduated from MIT in 1999'))
print(extract_entities('people who graduated from university in Canada in 1999'))


{'PERSON': 'Mary', 'ORG': 'Google', 'DATE': '2010'}
{'ORG': 'MIT', 'DATE': '1999'}
{'GPE': 'Canada', 'DATE': '1999'}


**Assigning roles using spaCy's parser**

In this exercise you'll use spaCy's powerful syntax parser to assign roles to the entities in your users' messages. To do this, you'll define two functions, find_parent_item() and assign_colors(). In doing so, you'll use a parse tree to assign roles, similar to how Alan did in the video.

Recall that you can access the ancestors of a word using its .ancestors attribute.

In [304]:
# Create the document
doc = nlp("let's see that jacket in red, some blue jeans, and a neon hoverboard")

colors = ['black','red','blue','grey','green', 'neon']
items = ['shoes', 'handback', 'jacket', 'jeans','hoverboard']

In [291]:
#classify words as entity types or colurs
def entity_type(word):
    _type = None
    if word.text in colors:
        _type = "color"
    elif word.text in items:
        _type = "item"
    return _type

print(doc[6])
print(entity_type(doc[6]))

red
color


In [296]:
# Iterate over parents in parse tree until an item entity is found
def find_parent_item(word):
    # Iterate over the word's ancestors
    for parent in word.ancestors:
        # Check for an "item" entity
        if entity_type(parent) == "item":
            return parent.text
    return None

print(doc[6])
print("Ancestors: ",[x for x in doc[6].ancestors])
print(find_parent_item(doc[6]))

red
Ancestors:  [in, jacket, see, let]
jacket


In [306]:
# For all color entities, find their parent item
def assign_colors(doc):
    # Iterate over the document
    for word in doc:
        # Check for "color" entities
        if entity_type(word) == "color":
            # Find the parent of the color
            item = find_parent_item(word)
            print("item: {0} has color : {1}".format(item, word))

# Assign the colors
assign_colors(doc)

item: jacket has color : red
item: jeans has color : blue
item: hoverboard has color : neon


### Robust language understand with Rasu NLU

- Rasa provides high level API for intent recognition and entity extraction
- its based on spaCy, scikit-learn, and other libs
- built in support for chatbot specific tasks

**Using Rasa:**
- provide trianing data in a json file (json format is based on key value pairs)
- training data should have **a list of dictionaries called training examples**. Each list contains:
    - example message
    - message intent
    - list of entites in message
- you can convert one of these training examples to readable formats using the json.dumps function:

        In [1]: from rasa_nlu.converters import load_data
        In [2]: training_data = load_data("./training_data.json")
        In [3]: import json

        In [4]: print(json.dumps(data.training_examples[22], indent=2))
        Out[4]: {
          "text": "i'm looking for a place in the north of town",
          "intent": "restaurant_search",
          "entities": [
            {
              "start": 31,
              "end": 36,
              "value": "north",
              "entity": "location"}]}

**Using rasa with Python:**
- To use rasa nlu in python you use an interpreter object
- this contains your trained models for intents and entities
- to use it, pass a message through the interpreters parse function, like so:

        In [1]: message = "I want to book a flight to London"
        In [2]: interpreter.parse(message))
        Out[2]: {
          "intent": {
            "name": "flight_search",
            "confidence": 0.9
          },
          "entities": [
            {
              "entity": "location",
              "value": "London",
              "start": 27,
              "end": 33}]}
        
**Create an interpreter:**

        In [1]: from rasa_nlu.config import RasaNLUConfig

        In [2]: from rasa_nlu.model import Trainer
        In [3]: config = RasaNLUConfig(cmdline_args={"pipeline": "spacy_sklearn"})
        In [4]: trainer = Trainer(config)
        In [5]: interpreter = trainer.train(training_data)

**Rasa pipeline (as seen in the RasaNLUconfig cmdline_args above):**
- A rasa pipeline is a list of components that will be used to process text. The spacy_sklearn components uses both of thes packages to process test with the following steps:

        In [1]: MIKES_PIPELINE = [
          "nlp_spacy",
          "ner_crf",
          "ner_synonyms", 
          "intent_featurizer_spacy",
          "intent_classifier_sklearn"]
          
    1. nlp_spacy: initializes the spacy english model ('en')
    2. ner_crf: uses conditional random field model to extract entities (CDF is a ML model that is good for identifying Entities, even with small datasets). 
    3. ner_synonms: maps entities with the same meaning to the same key (like New York City and NYC)
    4. intent_featurizer_spacy: creates vector representations of sentences (which takes average of individual spacy word vectors!)
    5. intent_classifier_sklearn: scikit learn SCV!

#### These two statements are identical:
-When defining a RasaNLUconfi object, you can either pass a predifed pipeline (like spacy_sklearn), or define a list of components you want to use, as in below.

        In [2]: RasaNLUConfig(cmdline_args={"pipeline": MIKES_PIPELINE})

        In [3]: RasaNLUConfig(cmdline_args={"pipeline": "spacy_sklearn"})

### Rasa NLU Exercise:
In this exercise you'll use Rasa NLU to create an interpreter, which parses incoming user messages and returns a set of entities. **Your job is to train an interpreter using the MITIE entity recognition model in rasa NLU.**

<font color = red> Code below wont work because I dont have access to the training data. But the outputs are there for reference</font>

    # Import necessary modules
    from rasa_nlu.converters import load_data
    from rasa_nlu.config import RasaNLUConfig
    from rasa_nlu.model import Trainer

    # Create args dictionary
    args = {"pipeline": "spacy_sklearn"}

    # Create a configuration and trainer
    config = RasaNLUConfig(cmdline_args=args)
    trainer = Trainer(config)

    # Load the training data
    training_data = load_data("./training_data.json")

    # Create an interpreter by training the model
    interpreter = trainer.train(training_data)

    # Try it out
    print(interpreter.parse("I'm looking for a Mexican restaurant in the North of town")) 
    
    <script.py> output:
        Fitting 2 folds for each of 6 candidates, totalling 12 fits
        {'entities': [{'start': 18, 'end': 25, 'extractor': 'ner_crf', 'entity': 'cuisine', 'value': 'Mexican'}, {'start': 44, 'end': 49, 'extractor': 'ner_crf', 'entity': 'location', 'value': 'North'}], 'text': "I'm looking for a Mexican restaurant in the North of town", 'intent_ranking': [{'confidence': 0.5710798636909156, 'name': 'restaurant_search'}, {'confidence': 0.17661071712356397, 'name': 'goodbye'}, {'confidence': 0.14328579605798059, 'name': 'affirm'}, {'confidence': 0.10902362312753977, 'name': 'greet'}], 'intent': {'confidence': 0.5710798636909156, 'name': 'restaurant_search'}}


<font color=green size=5> <b>3. Virtual Assistants and Accessing Data</font>

**A few notes on SQL injection:**
- When using sqlite3, dont use .format or other traditional string operations to execute sql commands. 
- The safe way to pass params is to add them as an extra tuple arguments to the execute function as seen below. The execute function has safeguards implemented to make sure malitious code cant be inject into our queries :


    # Bad Idea
    query = "SELECT name from restaurant where area='{}'".format(area)
    c.execute(query)
    
    # Better
    t = (area,price)
    c.execute('SELECT * FROM hotels WHERE area=? and price=?', t)